In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.utils import np_utils
from pandas import Series , DataFrame, Panel
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping
from keras import metrics
import plotly.plotly as py
import plotly.tools as tls

Using TensorFlow backend.


In [2]:
train = pd.read_csv('./input/train.csv')

TestX = (pd.read_csv('./input/test.csv').values).astype(float)
labels = train.ix[1:,0].values
TrainX = (train.ix[1:,1:].values).astype(float)
TrainY = np_utils.to_categorical(labels)

TrainX /= 255
TrainX -= np.std(TrainX)

C:\Users\willy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [10]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=256)
# pca.fit(TrainX)
# TrainX = pca.transform(TrainX)

# print(TrainX.shape, TrainY.shape)
print(TrainY)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


In [4]:
TrainX = TrainX.reshape(TrainX.shape[0],28,28, 1)

In [5]:
model = Sequential()
# 4x4 window
model.add(Conv2D(128, kernel_size=(8,8),
                 activation='relu',
                 input_shape=(28,28,1)))
# Dropout for regularization
model.add(Dropout(0.20))
# Reduce kernel size
model.add(Conv2D(64, kernel_size=(4, 4),activation='relu',))
model.add(Dropout(0.20))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Dropout(0.20))
model.add(Conv2D(64, kernel_size=(4, 4), activation='relu'))
model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(784, activation = 'relu'))
model.add(Dense(TrainY.shape[1],activation ='softmax'))
early_stopping_monitor = EarlyStopping(monitor="acc",min_delta=0.3,patience=2)

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

model.fit(TrainX,TrainY, epochs = 32, verbose = 2, batch_size = 512)

Epoch 1/32
37s - loss: 0.5228 - acc: 0.8544
Epoch 2/32
36s - loss: 0.0801 - acc: 0.9735
Epoch 3/32
43s - loss: 0.0484 - acc: 0.9850
Epoch 4/32
44s - loss: 0.0297 - acc: 0.9909
Epoch 5/32
47s - loss: 0.0220 - acc: 0.9929
Epoch 6/32
55s - loss: 0.0175 - acc: 0.9944
Epoch 7/32
63s - loss: 0.0132 - acc: 0.9958
Epoch 8/32
63s - loss: 0.0104 - acc: 0.9965
Epoch 9/32
63s - loss: 0.0084 - acc: 0.9973
Epoch 10/32
70s - loss: 0.0077 - acc: 0.9978
Epoch 11/32
62s - loss: 0.0069 - acc: 0.9981
Epoch 12/32
62s - loss: 0.0058 - acc: 0.9984
Epoch 13/32
63s - loss: 0.0068 - acc: 0.9981
Epoch 14/32
63s - loss: 0.0051 - acc: 0.9987
Epoch 15/32
70s - loss: 0.0061 - acc: 0.9983
Epoch 16/32
62s - loss: 0.0051 - acc: 0.9985
Epoch 17/32
67s - loss: 0.0038 - acc: 0.9989
Epoch 18/32
66s - loss: 0.0049 - acc: 0.9989
Epoch 19/32
71s - loss: 0.0047 - acc: 0.9988
Epoch 20/32
63s - loss: 0.0033 - acc: 0.9994
Epoch 21/32
70s - loss: 0.0052 - acc: 0.9988
Epoch 22/32
69s - loss: 0.0045 - acc: 0.9991
Epoch 23/32
70s - l

In [6]:
# TestX /= 255
# TestX -= np.std(TrainX)
# TestX = pca.transform(TestX)
TestX= TestX.reshape(TestX.shape[0],28,28,1)
Prediction = model.predict_classes(TestX , verbose = 2)

In [8]:
pd.DataFrame({"ImageId": list(range(1,len(Prediction)+1)), "Label": Prediction}).to_csv("submission_RMS_3_layers.csv", index=False, header=True)